In [4]:
import numpy as np
import torch
from PIL import Image
from scipy.interpolate import interpn
from torchvision import transforms
from tqdm import tqdm
from utils.tensor_utils import *
import wandb
from torch.utils.data import random_split

from Dataset.AerialDataset import AerialDataset
from utils.metrics_utils import calculate_ssim, calculate_psnr

In [5]:
from torchvision.transforms.v2 import Compose, RandomVerticalFlip

from torchvision.transforms.v2 import RandomHorizontalFlip

lr_size = 64
hr_size = 256
dataset_dir = 'C:\\Users\\adria\\Desktop\\dataset_tfg'

transforms = Compose([
        RandomHorizontalFlip(0.2),
        RandomVerticalFlip(0.2)]
    )

dataset = AerialDataset(dataset_dir, lr_size, hr_size, data_augmentation=transforms, aux_sat_prob=0.5,
                            sat_dataset_path=dataset_dir + '\\satelite_dataset', return_pil=True)

_, _, test_dataset = random_split(dataset, [0.6, 0.2, 0.2], generator=torch.Generator().manual_seed(420))

In [5]:
def interpolation(image, objective_dim, method):
    # Calculo nuevas dimensiones
    height, width = image.shape[0], image.shape[1]
    new_width, new_height = objective_dim[0], objective_dim[1]
    new_image = np.zeros((new_height, new_width, image.shape[2]))

    # Generar cuadrículas para las coordenadas X e Y de la imagen original y la interpolada
    x = np.linspace(0, width - 1, width)
    y = np.linspace(0, height - 1, height)
    new_x = np.linspace(0, width - 1, new_width)
    new_y = np.linspace(0, height - 1, new_height)
    new_image = interpn((y, x), image, (new_y[:, None], new_x), method=method, bounds_error=False, fill_value=0)
    return new_image

In [11]:
for method in ["nearest", "bilinear", "bicubic"]:
    n_samples = 0
    ssim = 0     
    psnr = 0
    #wandb.login()
    #wandb.init(project="SR model benchmarking", name = method)
    print(method)
    for images in tqdm(test_dataset):
        lr = images["lr"]
        if method == "cubic":
            interpolated = lr.resize((hr_size,hr_size), Image.BICUBIC)
        elif method == "nearest":
            interpolated = lr.resize((hr_size,hr_size), Image.NEAREST)
        elif method == "bilinear":
            interpolated = lr.resize((hr_size,hr_size), Image.BILINEAR)
            
        hr = images["hr"]
        ssim += calculate_ssim(np.array(interpolated), np.array(hr))
        psnr += calculate_psnr(np.array(interpolated), np.array(hr))
        n_samples += 1
      
    #wandb.log({"ssim" : ssim / n_samples, "psnr" : psnr / n_samples})
    print(f"ssim: {ssim / n_samples} : {psnr / n_samples}")
    #wandb.finish()

nearest


100%|██████████| 770/770 [00:54<00:00, 14.25it/s]


ssim: 0.6818108445035599 : 27.613466606209414
bilinear


100%|██████████| 770/770 [00:54<00:00, 14.25it/s]


ssim: 0.6833442072910356 : 27.315385058563685
bicubic


100%|██████████| 770/770 [00:58<00:00, 13.11it/s]

ssim: 0.4588148758792212 : 19.33430059419867


In [6]:
test_img = PIL.Image.open('test pictures\\Amnesty POI-6-3-1.png')
interpolated = test_img.resize((hr_size,hr_size), Image.BICUBIC)
interpolated.show()
interpolated = test_img.resize((hr_size,hr_size), Image.NEAREST)
interpolated.show()
interpolated = test_img.resize((hr_size,hr_size), Image.BILINEAR)
interpolated.show()